In [3]:
import h5py
import numpy as np

with h5py.File('/home/liw324/code/Segment/EarthVQA/log/sfpnr50/test_features/0.hdf5', 'r') as f:
    print(f.keys())
    print(f['feature'])
    # print(f['feature'][0])
    # print(f['feature'][0].shape)
    print(f['pred_mask'])
    # x = f['pred_mask'][:]
    # print(type(x))

    

<KeysViewHDF5 ['feature', 'pred_mask']>
<HDF5 dataset "feature": shape (2048, 32, 32), type "<f4">
<HDF5 dataset "pred_mask": shape (1024, 1024), type "|u1">


In [4]:
import numpy as np
import PIL.Image as Image

pic = Image.open("/data/likai/RSCMQA/target/6.png")
pic_numpy = np.array(pic)
print(pic_numpy)
print(np.unique(pic_numpy))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[  0 255]


In [ ]:
import os
import h5py
import numpy as np
from PIL import Image

# 定义颜色映射（0~9）
COLOR_MAP = {
    0: (0, 0, 0),              # nothing
    1: (255, 255, 255),        # Background
    2: (255, 0, 0),            # Building
    3: (255, 255, 0),          # Road
    4: (0, 0, 255),            # Water
    5: (159, 129, 183),        # Barren
    6: (0, 255, 0),            # Forest
    7: (255, 195, 128),        # Agricultural
    8: (165, 0, 165),          # Playground
    9: (0, 185, 246),          # Pond
}

def save_pred_masks_with_pil(hdf5_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    raw_dir = os.path.join(output_dir, 'raw')
    vis_dir = os.path.join(output_dir, 'vis')
    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(vis_dir, exist_ok=True)

    pred_mask_dict = {}

    for filename in os.listdir(hdf5_dir):
        if filename.endswith('.hdf5') or filename.endswith('.h5'):
            filepath = os.path.join(hdf5_dir, filename)
            with h5py.File(filepath, 'r') as f:
                if 'pred_mask' in f:
                    pred_mask = f['pred_mask'][:]
                    pred_mask_dict[filename] = pred_mask

                    # 保存原始mask为灰度图
                    raw_img = Image.fromarray(pred_mask.astype(np.uint8), mode='L')
                    raw_path = os.path.join(raw_dir, filename.replace('.hdf5', '.png').replace('.h5', '.png'))
                    # print(raw_path)
                    raw_img.save(raw_path)

                    # 生成彩色可视化图
                    h, w = pred_mask.shape
                    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
                    for label, color in COLOR_MAP.items():
                        color_mask[pred_mask == label] = color
                    vis_img = Image.fromarray(color_mask)
                    vis_path = os.path.join(vis_dir, filename.replace('.hdf5', '.png').replace('.h5', '.png'))
                    # print(vis_path)
                    vis_img.save(vis_path)
        # break

    return pred_mask_dict

mask_dict = save_pred_masks_with_pil('/home/liw324/code/Segment/EarthVQA/log/sfpnr50/test_features/test', '/home/liw324/code/Segment/EarthVQA/out')



/home/liw324/code/Segment/EarthVQA/out/raw/3169.png
/home/liw324/code/Segment/EarthVQA/out/vis/3169.png
/home/liw324/code/Segment/EarthVQA/out/raw/4108.png
/home/liw324/code/Segment/EarthVQA/out/vis/4108.png
/home/liw324/code/Segment/EarthVQA/out/raw/4027.png
/home/liw324/code/Segment/EarthVQA/out/vis/4027.png
/home/liw324/code/Segment/EarthVQA/out/raw/4143.png
/home/liw324/code/Segment/EarthVQA/out/vis/4143.png
/home/liw324/code/Segment/EarthVQA/out/raw/3122.png
/home/liw324/code/Segment/EarthVQA/out/vis/3122.png
/home/liw324/code/Segment/EarthVQA/out/raw/3046.png
/home/liw324/code/Segment/EarthVQA/out/vis/3046.png
/home/liw324/code/Segment/EarthVQA/out/raw/2740.png
/home/liw324/code/Segment/EarthVQA/out/vis/2740.png
/home/liw324/code/Segment/EarthVQA/out/raw/3966.png
/home/liw324/code/Segment/EarthVQA/out/vis/3966.png
/home/liw324/code/Segment/EarthVQA/out/raw/2624.png
/home/liw324/code/Segment/EarthVQA/out/vis/2624.png
/home/liw324/code/Segment/EarthVQA/out/raw/3802.png
/home/liw324

In [22]:
import os
import numpy as np
from PIL import Image

dir_name = '/home/liw324/code/Segment/EarthVQA/dataset/EarthVQA/Val/masks_png'
all_labels = set()

for path in os.listdir(dir_name):
    if not path.endswith('.png'):
        continue
    file_path = os.path.join(dir_name, path)
    mask = np.array(Image.open(file_path))
    unique_labels = np.unique(mask)
    all_labels.update(unique_labels)

print("All unique labels in the dataset:", sorted(all_labels))

All unique labels in the dataset: [0, 1, 2, 3, 4, 5, 6, 7, 8]


In [19]:
import os
import numpy as np
from PIL import Image
from collections import defaultdict

def compute_iou_per_class(pred, gt, num_classes):
    ious = []
    for cls in range(num_classes):
        pred_inds = (pred == cls)
        gt_inds = (gt == cls)
        intersection = np.logical_and(pred_inds, gt_inds).sum()
        union = np.logical_or(pred_inds, gt_inds).sum()
        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append(intersection / union)
    return ious

def evaluate_segmentation(gt_dir, pred_dir, num_classes=8):
    per_image_ious = {}
    total_ious = defaultdict(list)

    for filename in os.listdir(gt_dir):
        if not filename.endswith('.png'):
            continue
        gt_path = os.path.join(gt_dir, filename)
        pred_path = os.path.join(pred_dir, filename)
        if not os.path.exists(pred_path):
            print(f"[Warning] Prediction missing for {filename}")
            continue

        gt = np.array(Image.open(gt_path))
        pred = np.array(Image.open(pred_path))

        if gt.shape != pred.shape:
            print(f"[Error] Shape mismatch: {filename}")
            continue

        ious = compute_iou_per_class(pred, gt, num_classes)
        per_image_ious[filename] = ious
        for cls, iou in enumerate(ious):
            if not np.isnan(iou):
                total_ious[cls].append(iou)

    class_mious = {cls: np.mean(total_ious[cls]) if total_ious[cls] else float('nan') 
                   for cls in range(num_classes)}
    miou = np.nanmean(list(class_mious.values()))

    return per_image_ious, class_mious, miou

# 示例使用（路径需替换为你的）
per_image_ious, class_mious, miou = evaluate_segmentation('/home/liw324/code/Segment/EarthVQA/dataset/EarthVQA/Val/masks_png', 
                                                          '/home/liw324/code/Segment/EarthVQA/out/raw', 
                                                          num_classes=9)

In [24]:
from pprint import pprint

# pprint(per_image_ious)  # 每张图的IoU
# pprint(class_mious)     # 每个类别的mIoU
# print("mIoU =", miou)   # 所有类别的平均mIoU
print(per_image_ious.keys())

dict_keys(['4174.png', '3609.png', '3268.png', '3751.png', '3330.png', '3497.png', '2902.png', '2570.png', '3834.png', '3446.png', '3780.png', '3027.png', '2667.png', '3562.png', '3910.png', '4108.png', '3103.png', '2743.png', '2792.png', '3675.png', '3214.png', '2826.png', '4081.png', '3899.png', '3848.png', '4050.png', '3917.png', '3565.png', '2744.png', '3104.png', '3672.png', '2795.png', '2821.png', '3213.png', '4086.png', '2979.png', '4057.png', '3519.png', '2738.png', '3178.png', '4173.png', '3756.png', '2577.png', '2905.png', '3490.png', '3337.png', '3441.png', '3833.png', '2660.png', '3020.png', '3787.png', '3919.png', '4101.png', '3724.png', '3083.png', '4088.png', '3345.png', '3890.png', '2977.png', '3394.png', '3433.png', '3841.png', '4059.png', '3052.png', '2612.png', '2882.png', '3965.png', '3517.png', '3176.png', '2736.png', '3600.png', '3261.png', '2853.png', '3758.png', '3339.png', '2579.png', '3789.png', '4025.png', '3510.png', '3962.png', '2885.png', '2731.png', '3171

In [27]:
import pandas as pd

df = pd.DataFrame.from_dict(per_image_ious, orient='index')

# 可选：重命名列
df.columns = [f'class_{i}' for i in range(df.shape[1])]
df = df.fillna(0.0)

# 查看前几行
print(df.head())

          class_0   class_1   class_2   class_3   class_4   class_5   class_6  \
4174.png      0.0  0.196414  0.637512  0.509716  0.406242  0.036189  0.520264   
3609.png      0.0  0.615002  0.796056  0.794879  0.456171  0.365375  0.538353   
3268.png      0.0  0.407906  0.000000  0.104779  0.673776  0.000000  0.000000   
3751.png      0.0  0.000000  0.000000  0.000000  0.999873  0.000000  0.000000   
3330.png      0.0  0.225387  0.000000  0.000000  0.000000  0.000000  0.000000   

           class_7  class_8  
4174.png  0.656632      0.0  
3609.png  0.045181      0.0  
3268.png  0.006687      0.0  
3751.png  0.000000      0.0  
3330.png  0.010155      0.0  


In [29]:
df.to_csv('iou_SOBA.csv', index=True)